# Ada Boost Classifier

In [1]:
# Imports
import os, sys
import numpy as np
import matplotlib.pyplot as plt



# to enable local imports
module_path = os.path.abspath('../code')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from machine_learning_load_data import loadOnlineEEGdata

/home/nvidia/masterthesis/thesis_eeg/code


In [2]:
# Load some online EEG Data
eegData, freqData, entropyData = loadOnlineEEGdata(dirPath='../../EEG_Data/eeg_data_online', splitData=True)
eegX_train, eegy_train, eegX_test, eegy_test = eegData
freqX_train, freqy_train, freqX_test, freqy_test = freqData
X_train_entropy, y_train_entropy, X_test_entropy, y_test_entropy = entropyData

# reshape
freqX_train = freqX_train.reshape(freqX_train.shape[0], freqX_train.shape[2])
freqX_test = freqX_test.reshape(freqX_test.shape[0], freqX_test.shape[2])

X_train_entropy = X_train_entropy.reshape(X_train_entropy.shape[0], X_train_entropy.shape[2])
X_test_entropy = X_test_entropy.reshape(X_test_entropy.shape[0], X_test_entropy.shape[2])

Loading Online EEG Data from ../../EEG_Data/eeg_data_online ...
EEG Data Shape:
(5024, 512, 40) (5024,) (2154, 512, 40) (2154,)
Freq Data Shape:
(1008, 1, 1200) (1008,) (432, 1, 1200) (432,)
Entropy Data Shape:
(5024, 1, 200) (5024,) (2154, 1, 200) (2154,)


In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
f1_scorer = make_scorer(f1_score) 

# Create a model to test
model = AdaBoostClassifier()

# Create a parameter grid - here you specifiy which combinations you want to test
param_grid = [
    {'n_estimators': [500, 1000],
     'learning_rate' : [1, 1.2,]
    }
]

kFoldTimes = 8

# create a grid search
grid_search = GridSearchCV(model, param_grid, cv=kFoldTimes,
                            scoring=f1_scorer,
                            return_train_score=True,
                            n_jobs=-1)

# fit it with the data
result = grid_search.fit(X_train_entropy, y_train_entropy)

In [8]:
print("Best Params: {}".format(grid_search.best_params_))
print("Best Estimator: {}".format(grid_search.best_estimator_))

cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

Best Params: {'learning_rate': 1.2, 'n_estimators': 1000}
Best Estimator: AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.2,
                   n_estimators=1000, random_state=None)
0.6814987904093085 {'learning_rate': 1, 'n_estimators': 500}
0.679406349296515 {'learning_rate': 1, 'n_estimators': 1000}
0.6891893546976744 {'learning_rate': 1.2, 'n_estimators': 500}
0.6910805880717243 {'learning_rate': 1.2, 'n_estimators': 1000}


In [10]:
grid_search.best_estimator_

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.2,
                   n_estimators=1000, random_state=None)